<a href="https://colab.research.google.com/github/StuteePatil/American-Sign-Language/blob/main/ASL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
file = files.upload()
!unzip dataset.zip

In [ ]:
import os
import matplotlib.image as mpimg

images = []
labels = []

# creating dataset of images and their corresponding labels
def load_images(folder, label):
    for filename in os.listdir(folder):
        img = mpimg.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
            labels.append(label)
    return images

folder = 'asl_dataset/asl_dataset'

# accessing images for each sign from respective folders
for fname in os.listdir(folder):
    img_path = os.path.join(folder, fname)
    images = load_images(img_path, fname)

In [ ]:
import keras
import pandas as pd
import numpy as np

# create dataframe to store images and their corresponding labels
df = pd.DataFrame()
df['images'] = images
df['labels'] = labels

# convert labels (type = str) into categorical variables (type = int) 
# and store in a new dataframe column named 'target'
df['labels'] = df['labels'].astype('category')
df['target'] = df['labels'].cat.codes

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

# convert integer labels to one-hot encoding vector
enc = enc.fit_transform(df[['target']])
enc = enc.toarray()


In [ ]:
from sklearn.model_selection import train_test_split

X = np.array(images)
y = enc

# split the dataset into train, validation and test sets
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size = 0.3, stratify = y)
X_val, X_tt, y_val, y_tt = train_test_split(X_test, y_test, test_size = 0.5, stratify = y_test)


In [ ]:
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam, Adadelta
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.metrics import Accuracy

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

# CNN model for image classification
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=X_tr[0].shape))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32, (3,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(36, activation='softmax'))
opt = keras.optimizers.Adadelta(learning_rate = 0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(X_tr, y_tr, batch_size=100, epochs=100, verbose=1, validation_data=(X_val, y_val), callbacks=[earlystopping])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 398, 398, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 199, 199, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 197, 197, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 98, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 307328)            0         
_________________________________________________________________
dense (Dense)                (None, 36)                11063844  
Total params: 11,068,932
Trainable params: 11,068,932
Non-trainable params: 0
____________________________________________

In [ ]:
# predicting target value for test data
pred = model.predict(X_tt)

y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(y_tt, axis=1)

# evaluating prediction accuracy on test data
count=0
for i in range(len(y_pred)):
    if y_pred[i]==y_true[i]:
        count+=1

print('Accuracy: {}%' .format(round((count*100)/len(y_pred), 2)))

Accuracy: 93.39%
